In [0]:
%sql
-- Top Product by Year
CREATE OR REPLACE FUNCTION workspace.gold.fn_top_product_by_year(
    year INT
)
RETURNS TABLE (
    product_name STRING,
    total_quantity INT,
    total_revenue DOUBLE
)
RETURN
    SELECT 
        dp.product_name,
        SUM(fs.quantity) AS total_quantity,
        SUM(fs.sales_amount) AS total_revenue
    FROM workspace.gold.fact_sales fs
    JOIN workspace.gold.dim_products dp
        ON fs.product_key = dp.product_key
    WHERE YEAR(fs.order_date) = year
    GROUP BY dp.product_name
    ORDER BY total_quantity DESC
    LIMIT 1;

-- Top Month by Revenue
CREATE OR REPLACE FUNCTION workspace.gold.fn_top_month_by_revenue(
    year INT
)
RETURNS TABLE (
    month DATE,
    revenue DOUBLE
)
RETURN
    SELECT 
        date_trunc('month', fs.order_date) AS month,
        SUM(fs.sales_amount) AS revenue
    FROM workspace.gold.fact_sales fs
    WHERE YEAR(fs.order_date) = year
    GROUP BY month
    ORDER BY revenue DESC
    LIMIT 1;

-- Top customer by Revenue in a Year
CREATE OR REPLACE FUNCTION workspace.gold.fn_top_customer_by_year(
    year INT
)
RETURNS TABLE (
    customer_name STRING,
    total_revenue DOUBLE
)
RETURN
    SELECT 
        CONCAT(dc.first_name, ' ', dc.last_name) AS customer_name,
        SUM(fs.sales_amount) AS total_revenue
    FROM workspace.gold.fact_sales fs
    JOIN workspace.gold.dim_customers dc
        ON fs.customer_key = dc.customer_key
    WHERE YEAR(fs.order_date) = year
    GROUP BY dc.first_name, dc.last_name
    ORDER BY total_revenue DESC
    LIMIT 1;